**Imported Libraries & Setup**

In [ ]:

import nest_asyncio
nest_asyncio.apply()

import re
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

from textblob import TextBlob

import pandas as pd
pd.options.display.max_columns=50

import plotly.express as px
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))
tweets = pd.read_csv('PSUTweets.csv')

In [ ]:
# View Uploaded Data
tweets.head(5)

**Cleaning Dataset**

In [ ]:
def clean_text(text):
    #Any Patters Can be Used Here
    pattern1 = r'@[^ ]+'
    pattern2 = r'https?://[A-Za-z0-9./]+'
    pattern3 = r'\'s'
    pattern4 = r'\#\w+'
    pattern5 = r'&amp '
    pattern6 = r'[^A-Za-z\s]'

    combined_pattern = r'|'.join((pattern1,pattern2,pattern3,pattern4,pattern5,pattern6))
    text = re.sub(combined_pattern, "", text).lower()
    text = ' '.join(word for word in text.split() if word not in stopwords)
    return text.strip()

In [ ]:
#Cleaning Applied Here
tweets["cleaned_tweet"] = tweets["Tweet"].apply(clean_text)
#tweets["cleaned_tweet"]


**Processing Dataset | Sorting Based on Polarity and Subjectivity**

In [ ]:
for row in tweets.itertuples():
    tweet = tweets.at[row[0], 'cleaned_tweet']
#run sentiment using TextBlob
    analysis = TextBlob(tweet)
#set value to dataframe
    tweets.at[row[0], 'polarity'] = analysis.sentiment[0]
    tweets.at[row[0], 'subjectivity'] = analysis.sentiment[1]
#Scale Based on Polarity
    if analysis.sentiment[0]>0:
       tweets.at[row[0], 'Sentiment'] = "Positive"
    elif analysis.sentiment[0]<0:
       tweets.at[row[0], 'Sentiment'] = "Negative"
    else:
       tweets.at[row[0], 'Sentiment'] = "Neutral"



In [ ]:
tweet_polarity = tweets[["Date", "cleaned_tweet", "polarity", "Sentiment"]]
tweet_polarity

**Data Visulization: Sentiments Over Time**

In [ ]:
CountSentimentTotal = tweet_polarity.groupby(['Sentiment'])['Sentiment'].count().reset_index(name="Count")
CountSentimentTotal

In [ ]:
CountSentimentPerDay = tweet_polarity.groupby(['Sentiment', 'Date'])['Sentiment'].count().reset_index(name="Count")
CountSentimentPerDay

In [ ]:
fig1 = px.line(
CountSentimentPerDay,
x= "Date",
y = "Count",
title = "Sentiments on Twitter for PSU over time",
color = "Sentiment",
labels = {
    "date" : "Date",
    "Count" : "Number of Tweets"
},
color_discrete_sequence=px.colors.qualitative.Set1)

In [ ]:
fig1

NameError: ignored